In [1]:
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Dropout, Flatten, Input
from keras.optimizers import RMSprop
from keras.utils.np_utils import to_categorical
from PIL import Image
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import accuracy_score
from keras import backend as K

Using TensorFlow backend.


In [ ]:
#tuning activation function and compare
#only use training set to do training and validation

In [2]:
#training label
labels = np.load('labels.npy')
labels.shape

(33402, 62)

In [3]:
#img as train data
size = labels.shape[0]
numsample = 5
folder = 'train/croppedsampled/'
images = []

for k in range(numsample):
    for i in range(size):
        im = Image.open(folder+str(i+1)+'_'+str(k)+'.png')
        images.append(np.asarray(im))
        
images = np.array(images)
images.shape

(167010, 54, 54, 3)

In [4]:
#replicate labels to cover all sample
labels = np.tile(labels,(5,1))
labels.shape

(167010, 62)

In [5]:
#split training set and validation set
rs = ShuffleSplit(n_splits=1, test_size=.1, random_state=0)
train_index, test_index = next(rs.split(images)) #just 1
test_images = images[test_index]
test_labels = labels[test_index]
images = images[train_index]
labels = labels[train_index]

In [6]:
#initial model (same as initial model in other program)
a = Input(shape=(54, 54, 3))
#conv layer
c1 = Conv2D(64, 3,3, activation='relu', border_mode='same')(a)
c1p = MaxPooling2D(pool_size=(2, 2)) (c1)
c1d = Dropout(0.1)(c1p)

c2 = Conv2D(128, 3,3, activation='relu', border_mode='same')(c1d)
c2p = MaxPooling2D(pool_size=(2, 2)) (c2)
c2d = Dropout(0.1)(c2p)

c3 = Conv2D(192, 3,3, activation='relu', border_mode='same')(c2d)
c3p = MaxPooling2D(pool_size=(2, 2)) (c3)
c3d = Dropout(0.1)(c3p)


c4 = Conv2D(256, 3,3, activation='relu', border_mode='same')(c3d)
c4p = MaxPooling2D(pool_size=(2, 2)) (c4)
c4d = Dropout(0.1)(c4p)


c5 = Conv2D(256, 3,3, activation='relu', border_mode='same')(c4d)
c5p = MaxPooling2D(pool_size=(2, 2)) (c5)
c5d = Dropout(0.1)(c5p)

#linear layer
flat = Flatten()(c5d)
d1 = Dense(128, activation='relu')(flat)
d1d = Dropout(0.1)(d1)

#output
o_length = Dense(7, activation='softmax')(d1d)
o_digit1 = Dense(11, activation='softmax')(d1d)
o_digit2 = Dense(11, activation='softmax')(d1d)
o_digit3 = Dense(11, activation='softmax')(d1d)
o_digit4 = Dense(11, activation='softmax')(d1d)
o_digit5 = Dense(11, activation='softmax')(d1d)
model = Model(a, [o_length,o_digit1,o_digit2,o_digit3,o_digit4,o_digit5])

rms = RMSprop(lr=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=rms)


In [8]:
#training
result = model.fit(images, [labels[:,:7], labels[:,7:18], labels[:,18:29], labels[:,29:40], labels[:,40:51], labels[:,51:62]], 32, 20, validation_split=0.1, shuffle=True)

Train on 135278 samples, validate on 15031 samples
Epoch 1/20
135278/135278 [==============================] - 314s - loss: 5.7518 - dense_2_loss: 0.6318 - dense_3_loss: 1.8023 - dense_4_loss: 2.0212 - dense_5_loss: 0.9961 - dense_6_loss: 0.2168 - dense_7_loss: 0.0837 - val_loss: 3.6815 - val_dense_2_loss: 0.2836 - val_dense_3_loss: 1.1541 - val_dense_4_loss: 1.3667 - val_dense_5_loss: 0.7384 - val_dense_6_loss: 0.1372 - val_dense_7_loss: 0.0015
Epoch 2/20
135278/135278 [==============================] - 289s - loss: 3.3503 - dense_2_loss: 0.2717 - dense_3_loss: 1.0143 - dense_4_loss: 1.2177 - dense_5_loss: 0.6962 - dense_6_loss: 0.1467 - dense_7_loss: 0.0037 - val_loss: 2.5331 - val_dense_2_loss: 0.2103 - val_dense_3_loss: 0.7436 - val_dense_4_loss: 0.8995 - val_dense_5_loss: 0.5588 - val_dense_6_loss: 0.1197 - val_dense_7_loss: 0.0013
Epoch 3/20
135278/135278 [==============================] - 288s - loss: 2.4642 - dense_2_loss: 0.2053 - dense_3_loss: 0.7237 - dense_4_loss: 0.8566 - 

In [9]:
score = model.predict(test_images)

score = np.concatenate(score, axis=1)
roundedScore = np.zeros(score.shape, dtype="int32")
#choose threshold
roundedScore[score > 0.5] = 1

#rounded acc (exactly match)
accuracy_score(test_labels, roundedScore)

0.80288605472726182

In [10]:
#activation function = tanh (instead of relu in initial model)
a = Input(shape=(54, 54, 3))
#conv layer
c1 = Conv2D(64, 3,3, activation='tanh', border_mode='same')(a)
c1p = MaxPooling2D(pool_size=(2, 2)) (c1)
c1d = Dropout(0.1)(c1p)

c2 = Conv2D(128, 3,3, activation='tanh', border_mode='same')(c1d)
c2p = MaxPooling2D(pool_size=(2, 2)) (c2)
c2d = Dropout(0.1)(c2p)

c3 = Conv2D(192, 3,3, activation='tanh', border_mode='same')(c2d)
c3p = MaxPooling2D(pool_size=(2, 2)) (c3)
c3d = Dropout(0.1)(c3p)


c4 = Conv2D(256, 3,3, activation='tanh', border_mode='same')(c3d)
c4p = MaxPooling2D(pool_size=(2, 2)) (c4)
c4d = Dropout(0.1)(c4p)


c5 = Conv2D(256, 3,3, activation='tanh', border_mode='same')(c4d)
c5p = MaxPooling2D(pool_size=(2, 2)) (c5)
c5d = Dropout(0.1)(c5p)

#linear layer
flat = Flatten()(c5d)
d1 = Dense(128, activation='tanh')(flat)
d1d = Dropout(0.1)(d1)

#output
o_length = Dense(7, activation='softmax')(d1d)
o_digit1 = Dense(11, activation='softmax')(d1d)
o_digit2 = Dense(11, activation='softmax')(d1d)
o_digit3 = Dense(11, activation='softmax')(d1d)
o_digit4 = Dense(11, activation='softmax')(d1d)
o_digit5 = Dense(11, activation='softmax')(d1d)
model = Model(a, [o_length,o_digit1,o_digit2,o_digit3,o_digit4,o_digit5])

rms = RMSprop(lr=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=rms)


In [12]:
result = model.fit(images, [labels[:,:7], labels[:,7:18], labels[:,18:29], labels[:,29:40], labels[:,40:51], labels[:,51:62]], 32, 20, validation_split=0.1, shuffle=True)

Train on 135278 samples, validate on 15031 samples
Epoch 1/20
135278/135278 [==============================] - 286s - loss: 4.4195 - dense_9_loss: 0.4314 - dense_10_loss: 1.3758 - dense_11_loss: 1.6127 - dense_12_loss: 0.8230 - dense_13_loss: 0.1689 - dense_14_loss: 0.0077 - val_loss: 3.0475 - val_dense_9_loss: 0.2588 - val_dense_10_loss: 0.9018 - val_dense_11_loss: 1.0993 - val_dense_12_loss: 0.6564 - val_dense_13_loss: 0.1290 - val_dense_14_loss: 0.0022s: 0.4685 - dense_10_loss - ETA: 46s - loss: 4.6532 - dense_9_loss: 0.4636 - dense_10_loss: 1.4584 - dense_11_loss: 1.6964 - dense_12_loss: 0.85 - ETA: 43s - loss: 4.6347 - dense_9_loss: 0.4610 - dense_10_loss: 1.4519 - dense_11_loss: 1.6900 - dense_12_loss: 0.8510 - dense_13_loss:  - ETA: 41s - loss: 4.6243 - dense_9_loss: 0.4597 - dense_10_loss: 1.4478 - dense_11_loss: 1.6864 - dense_12_loss: 0.8495 - dense_13_loss: 0.1725 - dense_14 - ETA: 40s - loss: 4.6196 - dense_9_loss: 0.4591 - dense_10_loss: 1.4463 - dense_11_ - ETA: 35 - ETA:

135278/135278 [==============================] - 296s - loss: 0.6230 - dense_9_loss: 0.0461 - dense_10_loss: 0.2037 - dense_11_loss: 0.2002 - dense_12_loss: 0.1185 - dense_13_loss: 0.0533 - dense_14_loss: 0.0011 - val_loss: 0.7921 - val_dense_9_loss: 0.0682 - val_dense_10_loss: 0.2473 - val_dense_11_loss: 0.2543 - val_dense_12_loss: 0.1680 - val_dense_13_loss: 0.0536 - val_dense_14_loss: 8.0878e-04- dense - ETA: 124s - loss: 0.6090 - dense_9_loss: 0. - ETA: 108s - loss: 0.6106 - dense_9_loss: 0.0441 - dense_10_loss: 0.2004 - dense_11_loss: 0.1940 - dense_1 - ETA: 100s - loss: 0.6104 - dense_9_loss: 0.0441 - dense_10_loss: 0.2002 - dense_11_loss: 0.1949 - dense_12_loss: 0.1174 - dense_13_loss: 0.0529 - dense_14_loss: 8.7436e- - ETA: 100s - loss: 0.6104 - dense_9_loss: 0.0441 - dense_10_loss: 0.2002 - dense_11_loss: 0.1949 - dense_12_loss: 0.1174 - ETA: 97s - loss: 0.6114 - dense_9_loss: 0.0442 - dense_10_loss: 0.2008 - dense_11_loss: 0.1949 - dense_12_loss: 0.1176 - dense_13_loss: 0.053

In [13]:
score = model.predict(test_images)

score = np.concatenate(score, axis=1)
roundedScore = np.zeros(score.shape, dtype="int32")
#choose threshold
roundedScore[score > 0.5] = 1

#rounded acc (exactly match)
accuracy_score(test_labels, roundedScore)

0.78234836237351058

In [14]:
#activation function = sigmoid (instead of relu in initial model)
a = Input(shape=(54, 54, 3))
#conv layer
c1 = Conv2D(64, 3,3, activation='sigmoid', border_mode='same')(a)
c1p = MaxPooling2D(pool_size=(2, 2)) (c1)
c1d = Dropout(0.1)(c1p)

c2 = Conv2D(128, 3,3, activation='sigmoid', border_mode='same')(c1d)
c2p = MaxPooling2D(pool_size=(2, 2)) (c2)
c2d = Dropout(0.1)(c2p)

c3 = Conv2D(192, 3,3, activation='sigmoid', border_mode='same')(c2d)
c3p = MaxPooling2D(pool_size=(2, 2)) (c3)
c3d = Dropout(0.1)(c3p)


c4 = Conv2D(256, 3,3, activation='sigmoid', border_mode='same')(c3d)
c4p = MaxPooling2D(pool_size=(2, 2)) (c4)
c4d = Dropout(0.1)(c4p)


c5 = Conv2D(256, 3,3, activation='sigmoid', border_mode='same')(c4d)
c5p = MaxPooling2D(pool_size=(2, 2)) (c5)
c5d = Dropout(0.1)(c5p)

#linear layer
flat = Flatten()(c5d)
d1 = Dense(128, activation='sigmoid')(flat)
d1d = Dropout(0.1)(d1)

#output
o_length = Dense(7, activation='softmax')(d1d)
o_digit1 = Dense(11, activation='softmax')(d1d)
o_digit2 = Dense(11, activation='softmax')(d1d)
o_digit3 = Dense(11, activation='softmax')(d1d)
o_digit4 = Dense(11, activation='softmax')(d1d)
o_digit5 = Dense(11, activation='softmax')(d1d)
model = Model(a, [o_length,o_digit1,o_digit2,o_digit3,o_digit4,o_digit5])

rms = RMSprop(lr=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=rms)


In [16]:
result = model.fit(images, [labels[:,:7], labels[:,7:18], labels[:,18:29], labels[:,29:40], labels[:,40:51], labels[:,51:62]], 32, 20, validation_split=0.1, shuffle=True)

Train on 135278 samples, validate on 15031 samples
Epoch 1/20
135278/135278 [==============================] - 296s - loss: 6.5545 - dense_16_loss: 0.9654 - dense_17_loss: 2.0336 - dense_18_loss: 2.1914 - dense_19_loss: 1.1200 - dense_20_loss: 0.2164 - dense_21_loss: 0.0278 - val_loss: 6.0743 - val_dense_16_loss: 0.7854 - val_dense_17_loss: 1.9864 - val_dense_18_loss: 2.1201 - val_dense_19_loss: 1.0109 - val_dense_20_loss: 0.1695 - val_dense_21_loss: 0.0021oss: 9.9230 - dense_16_loss: 1.3482 - dense_17_loss: 2.3479 - dense_18_loss: 2.4523 - dense_19_loss: 2.1029 - dense_20_loss: 0.8333 - dense_21_loss: 0.838 - ETA: 349s - loss: 9.9070 - dense_16_loss: 1.3468 - dense_17_loss: 2.3470 - dense_18_loss: 2.4525 - dense_1 - ETA: 314s - loss: 8.9843 - dense_16_loss: 1.2939 - dense_17_loss: 2.2670 - dense_18_loss: 2.4135 - dense_19_loss: 1.772 - ETA: 300s - loss: 8.5572 - dense_16_loss: 1.2697 - dense_17_loss: 2.2285 - dense_18_loss: 2.3867 - dense_19_loss: 1.6352 - dense_20_loss: 0.5587 - dens

135278/135278 [==============================] - 269s - loss: 2.6136 - dense_16_loss: 0.1970 - dense_17_loss: 0.7687 - dense_18_loss: 0.9349 - dense_19_loss: 0.5831 - dense_20_loss: 0.1256 - dense_21_loss: 0.0043 - val_loss: 2.3102 - val_dense_16_loss: 0.1689 - val_dense_17_loss: 0.6654 - val_dense_18_loss: 0.8238 - val_dense_19_loss: 0.5382 - val_dense_20_loss: 0.1118 - val_dense_21_loss: 0.0021
Epoch 15/20
135278/135278 [==============================] - 268s - loss: 2.4788 - dense_16_loss: 0.1870 - dense_17_loss: 0.7274 - dense_18_loss: 0.8785 - dense_19_loss: 0.5597 - dense_20_loss: 0.1220 - dense_21_loss: 0.0043 - val_loss: 2.2034 - val_dense_16_loss: 0.1698 - val_dense_17_loss: 0.6276 - val_dense_18_loss: 0.7766 - val_dense_19_loss: 0.5170 - val_dense_20_loss: 0.1102 - val_dense_21_loss: 0.0021
Epoch 16/20
135278/135278 [==============================] - 268s - loss: 2.3541 - dense_16_loss: 0.1799 - dense_17_loss: 0.6859 - dense_18_loss: 0.8282 - dense_19_loss: 0.5359 - dense_20_

In [17]:
score = model.predict(test_images)

score = np.concatenate(score, axis=1)
roundedScore = np.zeros(score.shape, dtype="int32")
#choose threshold
roundedScore[score > 0.5] = 1

#rounded acc (exactly match)
accuracy_score(test_labels, roundedScore)

0.53068678522244173